In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.nn.utils.rnn import pad_sequence
import pickle
from torch.nn.functional import pad

ModuleNotFoundError: No module named 'A_Ising'

In [2]:
# Define the Transformer model for classification with padding and masking
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.model_dim = model_dim
        
        # Embedding layer to convert input to model dimensions
        self.embedding = nn.Linear(input_dim, model_dim)
        
        # Positional encoding for variable length sequences
        self.pos_encoder = PositionalEncoding(model_dim, dropout)
        
        # Transformer encoder
        encoder_layers = TransformerEncoderLayer(model_dim, num_heads, model_dim, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers)
        
        # Classification head
        self.fc_out = nn.Linear(model_dim, num_classes)  # Output layer for classification

    def forward(self, src, src_mask):
        src = self.embedding(src) * torch.sqrt(torch.tensor(self.model_dim, dtype=torch.float32))
        src = self.pos_encoder(src)
        transformer_output = self.transformer_encoder(src, src_mask)
        
        # Aggregate the output of the transformer (e.g., using mean pooling or just the first token)
        pooled_output = transformer_output.mean(dim=0)
        
        # Pass through classification layer
        output = self.fc_out(pooled_output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, model_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / model_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Example usage with variable-length sequences:
input_dim = 12  # Number of features
model_dim = 12  # Transformer model dimension
num_heads = 4   # Number of attention heads
num_layers = 3  # Number of transformer layers
num_classes = 2  # Number of classes for classification
dropout = 0.1   # Dropout rate

# Example of a batch of sequences with different lengths
seq1 = torch.randn(8, 10)  # 8 time steps, 10 features
seq2 = torch.randn(5, 10)  # 5 time steps, 10 features
seq3 = torch.randn(7, 10)  # 7 time steps, 10 features

# Pad the sequences to match the longest sequence length
padded_sequences = pad_sequence([seq1, seq2, seq3], batch_first=False)  # Shape: [max_seq_len, batch_size, input_dim]

# Create a mask to ignore the padded positions
def create_mask(padded_seqs):
    # Create a binary mask where 1 indicates valid tokens and 0 indicates padding
    return (padded_seqs != 0).float()

src_mask = create_mask(padded_sequences[:, :, 0])  # Shape: [max_seq_len, batch_size]

# Initialize the transformer classification model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 12)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 128, 256)     3328        ['input_1[0][0]']                
                                                                                                  
 layer_normalization (LayerNorm  (None, 128, 256)    512         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 128, 256)    1051904     ['layer_normalization[0][0]',

In [ ]:
model = TransformerClassifier(input_dim, model_dim, num_heads, num_layers, num_classes, dropout)

# Forward pass with masking
output = model(padded_sequences, src_mask)
print(output.shape)  # Output will have shape: [batch_size, num_classes]

# Example of using CrossEntropyLoss for training
criterion = nn.CrossEntropyLoss()
labels = torch.randint(0, num_classes, (padded_sequences.size(1),))  # Random labels for testing
loss = criterion(output, labels)
print(f"Loss: {loss.item()}")

In [67]:
target_size = 15
with open("Train_Data_15", 'rb') as file:
    raw_data = pickle.load(file)
raw_data

,System,System_cg,System_ews,Magnetization,Heat_capacity,Tc,Tbounds,null
0,"[[[-1.0, 1.0, -1.0, -1.0, -0.3884, -1.0, -1.0,...","[[[-0.1378530612244898, 0.14736326530612243, -...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.018412698412698412, 0.018412698412698412, 0...","[0.33963944335366997, 0.3579124328155069, 0.35...",40.875444,"[62.606506486380326, 52.452706542391226]",1
1,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.9922222222222222, 0.9861111111111112, 0.988...","[1.2372017981419958, 1.3653345443721225, 1.297...",24.084911,"[11.616837246892846, 18.783102486882193]",1
2,"[[[-1.0, -0.6432, -1.0, -1.0, 0.9332, 1.0, 0.5...","[[[-0.163825, -0.138975, 0.0859, -0.0919249999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.012222222222222223, -0.0033333333333333335...","[0.31869514155341827, 0.3215510437136325, 0.32...",11.122186,"[17.55984521695178, 15.251016519053056]",1
3,"[[[1.0, 1.0, 1.0, 0.404, 1.0, 1.0, 1.0, 1.0, 1...","[[[0.9844, 0.9337777777777778, 1.0, 1.0, 1.0, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.9792592592592593, 0.9822222222222222, 0.971...","[0.7742778242394478, 0.7550195841953804, 0.859...",3.977646,"[2.5002202103401916, 3.977646440808179]",0
4,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.9866666666666667, 0.9833333333333333, 0.983...","[0.3144528813989275, 0.3613046362387088, 0.374...",5.388354,"[3.5310727194934532, 2.3656714311515787]",1
5,"[[[-1.0, 0.7744, 1.0, 0.9052, -1.0, -1.0, -1.0...","[[[0.019600000000000006, -0.004777777777777757...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.02074074074074074, 0.007901234567901235, 0....","[0.7464312119787391, 0.7356450914805108, 0.747...",24.306963,"[37.41898618920932, 24.30696310131024]",0
6,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99115102040...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.9912925170068028, 0.9847619047619047, 0.981...","[0.5813449320773951, 0.6639666250178324, 0.700...",7.069180,"[4.581194000157167, 3.9339484262111943]",1
7,"[[[0.4412, 1.0, 1.0, 0.4676, 0.1076, 0.5796, -...","[[[0.2405, 0.1734, 0.4024111111111111, -0.1209...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.018271604938271607, 0.008148148148148147, ...","[0.49213712640097385, 0.5003961170853308, 0.50...",4.797242,"[6.274745527578487, 7.580653707229478]",1
8,"[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.9992743764172336, 0.999092970521542, 0.9989...","[0.3901518276599921, 0.3970269147229335, 0.402...",9.386998,"[4.361125586955422, 6.772710006220633]",1
9,"[[[-0.1364, 0.2336, -1.0, -1.0, -1.0, -0.3244,...","[[[0.0974222222222222, 0.0919555555555556, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.05185185185185185, -0.0854320987654321, -0...","[0.41895332146957903, 0.46079408881288497, 0.4...",5.738842,"[8.206871856128657, 8.014981605810522]",1


In [70]:
for thing in raw_data["System_ews"]:
    print(thing.shape,type(thing))

(549, 12) <class 'numpy.ndarray'>
(574, 12) <class 'numpy.ndarray'>
(573, 12) <class 'numpy.ndarray'>
(580, 12) <class 'numpy.ndarray'>
(570, 12) <class 'numpy.ndarray'>
(50, 12) <class 'numpy.ndarray'>
(550, 12) <class 'numpy.ndarray'>
(533, 12) <class 'numpy.ndarray'>
(530, 12) <class 'numpy.ndarray'>
(475, 12) <class 'numpy.ndarray'>


In [77]:
raw_data["System_ews"][5].shape, raw_data["System_cg"][5].shape

((50, 12), (50, 15, 15))

In [72]:
seq = [torch.from_numpy(run) for run in raw_data["System_ews"]]
torch.equal(seq[3][0],torch.zeros(12))

True

In [78]:
seq = [torch.from_numpy(run) for run in raw_data["System_ews"]]
seq_trunc = []
for run in seq:
    t = 0
    try:
        while torch.equal(run[t],torch.zeros(12)):
            t += 1
        seq_trunc.append(run[t:,:])
    except IndexError:
        pass

In [79]:
for thing in seq_trunc:
    print(thing.shape,type(thing))

torch.Size([438, 12]) <class 'torch.Tensor'>
torch.Size([445, 12]) <class 'torch.Tensor'>
torch.Size([435, 12]) <class 'torch.Tensor'>
torch.Size([408, 12]) <class 'torch.Tensor'>
torch.Size([460, 12]) <class 'torch.Tensor'>
torch.Size([408, 12]) <class 'torch.Tensor'>
torch.Size([399, 12]) <class 'torch.Tensor'>
torch.Size([422, 12]) <class 'torch.Tensor'>
torch.Size([285, 12]) <class 'torch.Tensor'>


In [66]:
torch.equal(seq[3][0],torch.zeros(target_size,target_size))

False

In [39]:
max_length = max ([thing.shape[0] for thing in raw_data["System_cg"]])
max_length

580

In [40]:
seq = [torch.from_numpy(thing) for thing in raw_data["System_cg"]]
news = []
for thing in seq:
    pad_amount = max_length - thing.shape[0]
    new_t = pad(thing, (0, 0, 0, 0, pad_amount, 0))
    news.append(new_t)


In [41]:
for thing in news:
    print(thing.shape,type(thing))

torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>
torch.Size([580, 15, 15]) <class 'torch.Tensor'>


In [42]:
news[0].shape

torch.Size([580, 15, 15])

In [58]:
target_size = 15
oner = news[0]
torch.equal(oner[300],torch.zeros(target_size,target_size))

False

In [51]:
oner.shape

torch.Size([580, 15, 15])